# School Data Investigation
### Alex Craig & Aaron Xie

## 0. Load Packages & Imports

In [44]:
import pandas as pd

## 0.1 Initial Data Loading

In [45]:

nerds_df = pd.read_excel("../data/MA_1819_final_January_28th_22.xlsx")
nerds_df.head()

,state,year,distid_stateassigned,schoolid_stateassigned,distname,schoolname,ncesdistid_admin,ncesdistid_geo,census_id,ncesid,...,pp_PD_cent_fed_raw_MA,pp_instrumaterial_cent_fed_raw_MA,pp_guidepsych_cent_fed_raw_MA,countenroll,noaudit,nositecen,adfinvars,distflex,flag_nerds,flag_f33
0,MA,2018-2019,445,105,Abby Kelley Foster Charter Public School,Abby Kelley Foster Charter Public School,2500051,2500051,NaN,250005101247,...,0,0,0,0,0,0,1,0,0,0
1,MA,2018-2019,1,1,Abington,Abington Early Education Program,2501650,2501650,2.230120e+13,250165002727,...,0,0,0,0,0,0,1,0,0,0
2,MA,2018-2019,1,505,Abington,Abington High,2501650,2501650,2.230120e+13,250165000001,...,0,0,0,0,0,0,1,0,0,0
3,MA,2018-2019,1,405,Abington,Abington Middle School,2501650,2501650,2.230120e+13,250165000002,...,0,0,0,0,0,0,1,0,0,0
4,MA,2018-2019,1,20,Abington,Beaver Brook Elementary,2501650,2501650,2.230120e+13,250165000434,...,0,0,0,0,0,0,1,0,0,0


In [46]:
naeps_df = pd.read_csv("../data/seda2022_admindist_poolsub_np_beta.csv")
naeps_df = naeps_df[naeps_df["stateabb"] == "MA"]
naeps_df.head()

,fips,stateabb,sedaadmin,sedaadminname,subject,subgroup,np_mn19_ol,np_mn19_ol_se,np_mn19_eb,np_mn19_eb_se,...,np_chg_ol,np_chg_ol_se,np_chg_eb,np_chg_eb_se,tot_asmt19,tot_asmt22,cell19,cell22,avg_asmt19,avg_asmt22
8495,25,MA,2500001,QUABBIN,mth,all,8.11,2.246520,8.51,1.86,...,-5.950,3.145586,-6.720,2.13,992,1007,6,6,165.3,167.8
8496,25,MA,2500001,QUABBIN,mth,nec,14.10,2.278977,13.80,1.78,...,-6.200,3.241386,-5.540,1.77,711,587,6,6,118.5,97.8
8497,25,MA,2500001,QUABBIN,mth,wht,8.86,2.222624,9.22,1.71,...,-6.260,3.122795,-6.800,1.65,904,888,6,6,150.7,148.0
8498,25,MA,2500001,QUABBIN,rla,all,8.95,2.288376,9.83,1.83,...,-1.860,3.290987,-3.470,1.92,993,1007,6,6,165.5,167.8
8499,25,MA,2500001,QUABBIN,rla,nec,15.10,2.454748,15.60,1.90,...,-0.114,3.581980,-0.906,1.65,711,587,6,6,118.5,97.8


In [47]:
racial_enrollment_df = pd.read_excel("../data/district_grade_race_enrollment_22.xlsx")
racial_enrollment_df.head()

,ORG_CODE,DISTRICT_NAME,COUNTY,Grade,District_Total,Black,Asian,Hispanic,Multiracial,NativeAmerican,NativeHawaiianPacificIslander,White
0,0001,Abington,Plymouth,Gr.1,186.0,10,4,26,10,0,0,136
1,0001,Abington,Plymouth,Gr.2,171.0,7,2,29,3,1,0,129
2,0001,Abington,Plymouth,Gr.3,166.0,8,7,14,5,2,0,130
3,0001,Abington,Plymouth,Gr.4,148.0,6,3,19,2,1,0,117
4,0001,Abington,Plymouth,Gr.5,169.0,7,4,22,2,2,0,132


In [48]:
socio_eco_df = pd.read_excel("../data/selectedpopulations.xlsx")
socio_eco_df.head()

,District Name,District Code,First Language Not English #,First Language Not English %,English Language Learner #,English Language Learner %,Students With Disabilities #,Students With Disabilities %,Low Income #,Low Income %,Free Lunch #,Free Lunch %,Reduced Lunch #,Reduced Lunch %,High Needs #,High Needs #.1,Economically Disadvantaged #,Economically Disadvantaged %
0,Abby Kelley Foster Charter Public (District),4450000,967,68.0,256,18.0,166,11.7,837,58.9,,,,,"1,005",70.7,,
1,Abington,10000,362,16.9,236,11.0,387,17.8,736,34.4,,,,,"1,029",47.3,,
2,Academy Of the Pacific Rim Charter Public (Dis...,4120000,158,33.8,44,9.4,115,24.6,273,58.5,,,,,332,71.1,,
3,Acton-Boxborough,6000000,"1,141",22.2,317,6.2,803,15.5,546,10.6,,,,,"1,538",29.6,,
4,Acushnet,30000,17,1.8,6,0.6,154,15.8,320,33.2,,,,,415,42.7,,


In [49]:
staff_race_gender = pd.read_excel("../data/staffracegender.xlsx")
staff_race_gender.head()

,District/School Name,District/School Code,African American (#),Asian (#),Hispanic (#),White (#),Native American (#),"Native Hawaiian, Pacific Islander (#)","Multi-Race,Non-Hispanic (#)",Females (#),Males (#),FTE Count
0,Abby Kelley Foster Charter Public (District),4450000,8.8,1.0,8.0,173.4,0.0,1.0,2.0,153.3,41.0,194.2
1,Abington,10000,2.0,2.0,9.4,266.2,0.0,0.0,1.0,235.0,44.6,280.6
2,Academy Of the Pacific Rim Charter Public (Dis...,4120000,26.5,8.2,9.0,50.2,0.0,0.0,1.0,62.9,32.0,94.9
3,Acton-Boxborough,6000000,5.0,41.4,14.6,718.1,1.0,0.0,5.0,678.5,106.5,785.1
4,Acushnet,30000,1.0,0.0,0.0,120.6,0.0,0.0,0.0,102.8,18.8,121.6
